In [ ]:
%pylab inline

import os
import sys
import socket

sys.path.append("/root")
sys.path.append("/home/peter")
sys.path.append("/home/peter/Desktop")
sys.path.append("/home/peter/Desktop/projects")
sys.path.append("/home/peter/Desktop/work")
sys.path.append("/home/peter/code/projects")
sys.path.append("/home/peter/code/work")
sys.path.append("/home/peter/code/common")

from ipywidgets import interact, interactive, fixed, IntSlider, FloatSlider
from IPython.display import clear_output, display, HTML

import cv2
import os
import pickle as pkl
from sklearn import linear_model
from sklearn.decomposition import PCA
import random as rnd
import numpy as np
import matplotlib.pyplot as plt

import theano.tensor as T
import theano as th

from aidevutil import *

import cymysql as sql
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import datetime
import time

In [ ]:
from sklearn.preprocessing import scale, robust_scale, normalize

In [ ]:
input_source = np.load(open('data_eth_1m_2019.npy','rb'))
to_predict = np.load(open('data_eth_1m_2019_targets.npy','rb'))

to_predict = to_predict[3,:].reshape(-1)

In [ ]:
input_source.shape

In [ ]:
input_source = input_source.T

In [ ]:
is_orig = np.copy(input_source)
cp = int(0.8*len(input_source))
test_input_source = input_source[cp:, :]
test_to_predict = to_predict[cp:]
input_source = input_source[0:cp, :]
to_predict = to_predict[0:cp]

In [ ]:
test_input_source = test_input_source.T
input_source = input_source.T

In [ ]:
input_source.shape, test_input_source.shape

In [ ]:
to_predict[0:50]

In [ ]:
winlen = 1

In [ ]:
sliding_window_jump = 1
predict_time_ahead = 1

In [ ]:
def get_partitioned_counts(cl, rr, rt):
    " data, steps_forward, partition_num"
    cl = cl[rr:] - cl[0:-rr]
    cl = hstack([[0]*rr, cl])
    pdown = len(cl[cl < -rt])
    pup = len(cl[cl > rt])
    pmid = len(cl[(cl <= rt) & (cl >= -rt)])
    return pup, pmid, pdown

In [ ]:
def partition_data_num(cl, predict_ahead):
    a=[]
    b=[]
    c=[]
    ts = np.linspace(0.001, 10.0, 100000, )
    for x in tqdm(ts):
        au, am, ad = get_partitioned_counts(cl, predict_ahead, x)
        a.append(au)
        b.append(am)
        c.append(ad)
    a, b, c = array(a), array(b), array(c)
    return ts[np.argmin(np.abs(a-b) + np.abs(a-c) + np.abs(b-c))]

In [ ]:
# create training set
sep = partition_data_num(to_predict, predict_time_ahead)
sep

In [ ]:
x = []
y = []

ups = []
downs = []
mids = []

for i in tqdm(range(0, input_source.shape[1]-(winlen+100), sliding_window_jump)):
    # form the input
    xs = input_source[:, i:i+winlen]
    #xs = scale(xs, axis=1)
    xs = xs.ravel()

    # for the output
    now = to_predict[i+winlen-1].ravel() # close
    future = to_predict[i+winlen+(predict_time_ahead-1)].ravel() # next close

    ys = future-now
    magn = abs(ys)

    if magn < sep:
        mids.append( (xs, np.array([0, 1, 0])) )
    else:
        if ys < 0:
            downs.append( (xs, np.array([0, 0, 1])) )
        else:
            ups.append( (xs, np.array([1, 0, 0])) )

    if (i%1000)==0:
        clear_output(wait=True)
        print('({}/{})'.format(i, input_source.shape[1]-winlen))

print('Ups: {0} Mids: {1} Downs: {2}'.format(len(ups), len(mids), len(downs)))
print('Separator: %3.5f' % sep)
print()

samplesize = min(len(ups), len(downs), len(mids))
s1 = rnd.sample(ups, samplesize)
s2 = rnd.sample(downs, samplesize)
s3 = rnd.sample(mids, samplesize)
a = s1 + s2 + s3
rnd.shuffle(a)
x = [x[0] for x in a]
y = [x[1] for x in a]

x = np.vstack(x)
y = np.vstack(y)

In [ ]:
x.shape, y.shape

In [ ]:
x.shape = (x.shape[0], -1)

In [ ]:
x.shape

In [ ]:
#matshow(x[400:500].T); colorbar();

In [ ]:
cutpoint = int(0.8 * x.shape[0])
x_train = x[0:cutpoint]
x_test = x[cutpoint:]
y_train = y[0:cutpoint]
y_test = y[cutpoint:]

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
# Classifiers
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier 
from sklearn.naive_bayes import GaussianNB
#from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
import sklearn as sk
import time

In [ ]:
clf = LogisticRegression()#GradientBoostingClassifier(n_estimators=100, loss='deviance')

In [ ]:
%%time

clf.fit(x_train, argmax(y_train, axis=1))

In [ ]:
clf.score(x_train, argmax(y_train, axis=1))

In [ ]:
clf.score(x_test, argmax(y_test, axis=1))

In [ ]:
# Here the model is saved
pkl.dump(clf, open('test.pkl','wb'))

In [ ]:
from tradesys import *
from tradesys.testing import *
from tradesys.tstools import time_range, timedelta

In [ ]:
df = pd.read_csv('gemini_ETHUSD_2019_1min.csv')
nd = []
for i,(d,o,h,l,c,v) in df[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']].iterrows():
    ds = d.split(' ')[0].split('-')
    hs = d.split(' ')[1].split(':')[:]
    ds = [int(x) for x in ds]
    hs = [int(x) for x in hs]
    nd.append(ds + hs + [o,h,l,c,v])
datamatrix = DataMatrix()
for y,mn,d,hr,m,s, o,h,l,c,v in nd[::-1]:
    datamatrix.add_row(DateTime(y,mn,d,hr,m,s), o,h,l,c,int(v))
data = datamatrix

In [ ]:
opts = BacktestOptions()
opts.commission = 0
opts.slippage = 0.0
opts.backtest_mode = BacktestMode.TRADESTATION
opts.market_open_time = Time(0,0)
opts.market_close_time = Time(0,0)

In [ ]:
s = """ 

A = MLSIGNAL(180, 1, modelname='test', datamodel='datamodel_super', modeltype='sklearn', dolog=true, noinput=false);

BUY_IF OUTSIDE_MARKET and (A == I(1));
SELL_IF OUTSIDE_MARKET and (A == I(-1));
CLOSE_IF BARS_SINCE_ENTRY >= I(0);

"""

In [ ]:
%%time

info = backtest(data, s, options=opts, plot=1, prt=1, )